In [68]:
!pip install -U selenium pandas openpyxl loguru tenacity python-dotenv

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------------ --------------------------- 3.4/11.0 MB 20.2 MB/s eta 0:00:01
   ----------------------------- ---------- 8.1/11.0 MB 21.9 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.2.3
    Uninstalling tenacity-8.2.3:
      Successfully uninstalled tenacity-8.2.3
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 0.21.0
    Uninstalling python-dotenv-0.21.0:
      Successfully uninstalled python-dotenv-0.21.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires tenacity<9,>=8.1.0, but you have tenacity 9.1.2 which is incompatible.


### Library Import

In [131]:
import time, random, re, sys
from pathlib import Path

import pandas as pd
from loguru import logger

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait #wait until the page is ready
from selenium.webdriver.support import expected_conditions as EC #wait until the page is ready

from urllib.parse import quote_plus #

#### Add Chrome browser window for python to browser excess

In [113]:
def make_driver(headless=True):
    options = Options()
    options.add_argument("--window-size=1920,1080")
    if headless:
        options.add_argument("--headless")  # Run browser without opening window
    driver = webdriver.Chrome(options=options)
    return driver

#### Make driver

In [126]:
driver = make_driver(headless=False)  # set to False to watch it open

##### Build search url

In [140]:
def search_url(query, page=1):
    return f"https://www.ebay.com/sch/i.html?_nkw={quote_plus(query)}&_pgn={page}"

In [171]:
def scrape_one_page(driver, query="iphone 13", page=1):
    url = search_url(query, page)
    driver.get(url)

    # wait until at least one product appears
    # WebDriverWait(driver, 5).until(
    #     EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.s-item"))
    # )

    cards = driver.find_elements(By.CSS_SELECTOR, "li.s-item")
    results = []

    for c in cards:
        try:
            title = c.find_element(By.CSS_SELECTOR, ".s-item__title").text
            price = c.find_element(By.CSS_SELECTOR, ".s-item__price").text
            link = c.find_element(By.CSS_SELECTOR, "a.s-item__link").get_attribute("href")

            results.append({
                "title": title,
                "price_raw": price,
                "link": link,
            })
        except Exception as e:
            print("Skipping one card:", e)

    return results

In [175]:
driver = make_driver(headless=False)
data = scrape_one_page(driver, "iphone 13", 1)
driver.quit()

print(len(data), "items found")
pd.DataFrame(data).head()

0 items found


""
